<a href="https://colab.research.google.com/github/dfu2020/ATMS-597-SP-2020-Project-2-Group-D/blob/master/Project_2_Group_D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import requests

def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        Response object.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'UZdWJQdqeSTqhkXeOlNYMGzbwUkFJAeh'
        },
        params=payload
    )

In [35]:
import datetime
from dateutil.relativedelta import relativedelta

from IPython import display # for updating the cell dynamically

current = datetime.date(1900, 7, 1)
end = datetime.date(2019, 7, 1)

results = []

while current < end:
    # update the cell with status information
    display.clear_output(wait=True)
    display.display(f'Gathering data for {str(current)}')
    
    response = make_request(
        'data', 
        {
            'datasetid' : 'GHCND', # Global Historical Climatology Network - Daily (GHCND) dataset
            'locationid' : 'CITY:US360019', # NYC
            'startdate' : current,
            'enddate' : current,
            'units' : 'metric',
            'limit' : 1000 # max allowed
        }
    )

    if response.ok:
        # we extend the list instead of appending to avoid getting a nested list
        results.extend(response.json()['results'])

    # update the current date to avoid an infinite loop
    current += relativedelta(years=1)

'Gathering data for 2018-07-01'

In [36]:
df = pd.DataFrame(results)
df.head()

,date,datatype,station,attributes,value
0,1900-07-01T00:00:00,PRCP,GHCND:USC00282644,"P,,6,1700",0.0
1,1900-07-01T00:00:00,TMAX,GHCND:USC00282644,",,6,1700",26.7
2,1900-07-01T00:00:00,TMIN,GHCND:USC00282644,",,6,1700",12.2
3,1900-07-01T00:00:00,PRCP,GHCND:USC00286775,"P,,6,0800",0.0
4,1900-07-01T00:00:00,TMAX,GHCND:USC00286775,",,6,0800",26.7


In [0]:
df.to_csv('nyc_weather_1900_2018.csv', index=False)

In [46]:
weather = pd.read_csv('nyc_weather_1900_2018.csv',index_col='date', parse_dates=['date'])
tobs_data = weather.query('datatype == "TMIN"')
tobs_data

,datatype,station,attributes,value
date,,,,
1900-07-01,TMIN,GHCND:USC00282644,",,6,1700",12.2
1900-07-01,TMIN,GHCND:USC00286775,",,6,0800",12.2
1900-07-01,TMIN,GHCND:USC00287079,",,6,1900",11.7
1900-07-01,TMIN,GHCND:USC00305540,",,6,",14.4
1900-07-01,TMIN,GHCND:USW00014734,",,Z,",12.8
...,...,...,...,...
2018-07-01,TMIN,GHCND:USW00054787,",,W,",22.8
2018-07-01,TMIN,GHCND:USW00094728,",,W,2400",26.1
2018-07-01,TMIN,GHCND:USW00094741,",,W,",24.4


In [49]:
mean_tmin = tobs_data.groupby('date').mean()
mean_tmin

,value
date,
1900-07-01,12.766667
1901-07-01,23.616667
1902-07-01,15.583333
1903-07-01,20.100000
1904-07-01,19.312500
...,...
2014-07-01,20.600000
2015-07-01,19.728571
2016-07-01,18.642857
